This `bash` notebook provides an example of *regridding* using `nco` and the Earth System Modeling Framework.    
Regridding consists in obtaining a given field on a new grid (finer or coarser) using spatial interpolation techniques.
# Requirements
To run this notebook you need:
* a recent version of [`nco`](http://nco.sourceforge.net/) toolbox
* the Earth System Modeling Framework [ESMF](https://www.earthsystemcog.org/projects/esmf/)

# Configuration

Check if `ncremap` is installed

In [12]:
which ncremap

/usr/local/bin/ncremap


Check if the LD_LIBRARY_PATH is properly defined:    
it has to include the path to the directory containing `libnco.so`

In [13]:
echo ${LD_LIBRARY_PATH}

:/usr/local/lib


If that's not the case, add it like this:

In [14]:
export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/usr/local/lib
echo ${LD_LIBRARY_PATH}

:/usr/local/lib:/usr/local/lib


Check if `ESMF_RegridWeightGen` is installed and in your PATH:

In [15]:
which ESMF_RegridWeightGen

/home/ctroupin/bin/ESMF_RegridWeightGen


# Input files
We will need two files:
1. A data file (`regrid_input.nc`) containing at least one field to be re-interpolated.
2. A grid file (`regrid_newgrid.nc`) storing the coordinates (longitude, latitude) of the new grid.     

These files are located in the `data` directory. In addition we provide `regrid_output_ref.nc` which stores the expected results of regridding, so that comparison can be performed.

In [16]:
ls ./data

regrid_input.nc  regrid_newgrid.nc  regrid_output_ref.nc


You can visualise the file content using `ncview` tool:

In [17]:
ncview data/regrid_input.nc

Ncview 2.1.6 David W. Pierce  29 Oct 2015
http://meteora.ucsd.edu:80/~pierce/ncview_home_page.html
Copyright (C) 1993 through 2015, David W. Pierce
Ncview comes with ABSOLUTELY NO WARRANTY; for details type `ncview -w'.
This is free software licensed under the Gnu General Public License version 3; type `ncview -c' for redistribution details.

calculating min and maxes for analyzed_field...
XIO:  fatal IO error 11 (Resource temporarily unavailable) on X server ":0"
      after 885 requests (885 known processed) with 0 events remaining.


: 1

## Modify dimension and variable names

The tool expects that the dimension are lon and lat, while our file has x and y.     
We will use [`ncrename`](https://linux.die.net/man/1/ncrename) to solve this.     
The will create the new files in an `output` directory.

In [19]:
mkdir -pv output
cp ./data/regrid_input.nc ./output/regrid_input_latlon.nc
ncrename -d x,lon -v x,lon -d y,lat -v y,lat ./output/regrid_input_latlon.nc

ncrename: In total renamed 0 attributes, 2 dimensions, 0 groups, and 2 variables


### Perform the regridding
It might take a few seconds...

In [21]:
ncremap -i ./output/regrid_input_latlon.nc -d ./data/regrid_newgrid.nc -o ./output/regrid_output.nc

Input #00: ./output/regrid_input_latlon.nc
Grid(src): /tmp/ncremap_tmp_grd_src.nc.pid10526
Grid(dst): /tmp/ncremap_tmp_grd_dst.nc.pid10526
Map/Wgt  : /tmp/ncremap_tmp_map_esmf_bilinear.nc.pid10526
Regridded: ./output/regrid_output.nc


Now we can check the dimensions of the newly created file:

In [22]:
ncdump -h ./output/regrid_output.nc | head -5

netcdf regrid_output {
dimensions:
	lat = 321 ;
	lon = 851 ;
	nbnd = 2 ;


and compare them to the original one:

In [23]:
ncdump -h ./data/regrid_input.nc | head -5

netcdf regrid_input {
dimensions:
	x = 426 ;
	y = 161 ;
variables:


## Final check

Let's compare with the reference file provided in the directory:

In [27]:
ncdiff ./output/regrid_output.nc ./data/regrid_output_ref.nc ./output/output_diff.nc

If everything went fine, the resulting field should only contain zeros.     
We show only the first lines of the `ncdump` operation:

In [29]:
ncdump -v analyzed_field ./output/output_diff.nc | head -200 | tail -30

    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Let's also check the field aspect using `ncview`:

In [30]:
ncview output/regrid_output.nc

Ncview 2.1.6 David W. Pierce  29 Oct 2015
http://meteora.ucsd.edu:80/~pierce/ncview_home_page.html
Copyright (C) 1993 through 2015, David W. Pierce
Ncview comes with ABSOLUTELY NO WARRANTY; for details type `ncview -w'.
This is free software licensed under the Gnu General Public License version 3; type `ncview -c' for redistribution details.

XIO:  fatal IO error 11 (Resource temporarily unavailable) on X server ":0"
      after 686 requests (686 known processed) with 0 events remaining.


: 1